# ITC experiment simulation

This notebook performs analyses described in the 

**Manuscript:**

Modelling the decamerisation cycle of PRDX1 and the inhibition-like effect on its peroxidase activity

**Authors:**

C. Barry, C. Pillay, J. Rohwer

**Purpose:**

Writes: Barranco-Medina 2008 Fig. 1a Arabidopsis thaliana injection parameters (filename: `ITC_At_inject_params.csv`)

Analysis: Demonstrate ITC model using published parameters i.e. against Barranco-Medina 2008 Fig. 1a

Analysis: Test different baseline correction methods

Plot: Figure 3

**Requirements:**

Python libraries (see **Imports** below)

ITC Prx dimer-decamer equilibrium model (`ITC_Prx_dim-dec.psc`)

##### Imports

In [ ]:
import os
import math

import scipy as sp
import numpy as np
from matplotlib import pyplot as plt
import pandas as pd
from sklearn.metrics import auc
from scipy.interpolate import interp1d

import pysces
pysces.enablePandas()


###### Get directory paths

In [ ]:
prev_dir = os.path.split(os.getcwd())[0]

mod_dir = os.path.join(prev_dir,"models")
if not os.path.isdir(mod_dir): os.mkdir(mod_dir) # ensure dir exists

par_dir = os.path.join(prev_dir,"params")
if not os.path.isdir(par_dir): os.mkdir(par_dir) # ensure dir exists
    
fig_dir = os.path.join(os.getcwd(),"Figures")
if not os.path.isdir(fig_dir): os.mkdir(fig_dir) # ensure dir exists


##### Matplotlib stuff

In [ ]:
%matplotlib inline

multiplier = 1
mpl_width = 4.5*multiplier
mpl_height = 3.5*multiplier
mpl_dpi = 600
mpl_xlabel_fontsize = "large"
mpl_ylabel_fontsize = "large"
mu = "\u03bc"

##### Coswave

In [ ]:
time_range = range(900)
injection_interval = 200 # period
coswave = np.array([np.cos(_TIME_*2*np.pi/(injection_interval)) for _TIME_ in time_range])
plt.plot(time_range, coswave)


## Model

### Inject params

In [ ]:
inj_enthalpy_At = 142*5 # kcal/mol = µcal/nmol of decamer(x5)
inj_volume_At = 1.6*10**(-3) # µl to ml
cell_volume_init = 1400*10**(-3) # µl to ml
inj_mon_At = 70 # µM MONOMERS
inj_interval_At = 180 # sec


Fig 1a of Barranco-Medina 2008 was the basis for injection parameters below.
In this figure, the bottom x-axis does not correspond to the Prx concentration indicated by the annotation.
The injection interval of the trace corresponds to the described methodology (180 s) but the measured enthalpy per mole of monomer is slightly higher than reported in their text (71 vs ~82 kcal/mol injectant).
Therefore, we assume that the Fig 1a trace was produced with a higher Prx in the syringe than the 70 µM described in the methodology.

The injected Prx concentration was calculated from the annotated Prx in the cell below.

In [ ]:
inj_to_arrow = 16 # no. injections
arrow_Prx = 1.52 # µM
dPrx_inj = arrow_Prx/inj_to_arrow
inj_mon_by_anno = dPrx_inj*cell_volume_init/inj_volume_At
print(f"Injected Prx monomer calculated by annotation: {inj_mon_by_anno:.3}")

Alternatively, the injected Prx of trace was calculated from the injected Prx reported in the text the ratio of dHm from the text and dHm read from the plot. 

In [ ]:
dHm_plot = 82
dHm_text = 71
inj_mon_by_ratio = inj_mon_At*dHm_plot/dHm_text
print(f"Injected Prx monomer calculated by ratio: {inj_mon_by_ratio:.3}")

In [ ]:
# Update values for injected Prx monomer, dimer, and decamer
inj_dim_est_At = 0.8*inj_volume_At # CTC and µM to nmol
inj_dec_est_At = ((inj_mon_by_anno/2 - 0.8)/5)*inj_volume_At  # Total - CTC and µM to nmol

In [ ]:
# Create df used to updAte models later
ITC_At_inject_params = {"Parameter": ["injection_enthalpy",
                                      "injection_volume",
                                      "injection_Prx_mon",
                                      "injection_dimer",
                                      "injection_decamer",
                                      "injection_interval",
                                      "first_inject_ratio",
                                      "cell_volume_init"],
         
                       "Value":[inj_enthalpy_At,
                                inj_volume_At,
                                inj_mon_At,
                                inj_dim_est_At, 
                                inj_dec_est_At,
                                inj_interval_At,
                                1,
                               cell_volume_init]}

df_ITC_At_inject_params = pd.DataFrame(ITC_At_inject_params)
df_ITC_At_inject_params.set_index("Parameter",inplace=True)
df_ITC_At_inject_params

In [ ]:
ITC_At_inject_params = dict(zip(ITC_At_inject_params["Parameter"],ITC_At_inject_params["Value"]))

In [ ]:
# Write to file
df_ITC_At_inject_params.to_csv(
    os.path.join(par_dir, "ITC_At_inject_params.csv"), sep=',', encoding='utf-8'
)

## ITC Prx dec model expo 130

In [ ]:
# Assign injection parameters
inj_params =  ITC_At_inject_params


In [ ]:
# ITC load model
# mod_ITC_Prx_dim_dec = pysces.model("ITC_testing_amts_1event.psc",dir = mod_dir)
mod_ITC_Prx_dim_dec = pysces.model("ITC_Prx_dim-dec.psc",dir = mod_dir)
# mod_ITC_Prx_dim_dec.enableDataPandas()
mod_ITC_Prx_dim_dec.mode_integrate_all_odes = True # As recommented by pysces
mod_ITC_Prx_dim_dec.__settings__["cvode_access_solver"] = False
mod_ITC_Prx_dim_dec.__settings__["cvode_return_event_timepoints"] = False
mod_ITC_Prx_dim_dec.SetQuiet()

# Set injection params
for parameter in inj_params.keys():
    value = inj_params[parameter]
    setattr(mod_ITC_Prx_dim_dec,parameter,value)

# Adjust to e=130
setattr(mod_ITC_Prx_dim_dec, "exponent", 130)
setattr(mod_ITC_Prx_dim_dec, "Kd_app", 2.4*10**(-10))

# Update with arbitrary rates for pretty plots
mod_ITC_Prx_dim_dec.koff = 0.005  # Arbitrary
mod_ITC_Prx_dim_dec.kon = mod_ITC_Prx_dim_dec.koff/mod_ITC_Prx_dim_dec.Kd_app

# Load syringe model
mod_syr = pysces.model("Prx_dim-dec.psc", dir = mod_dir)
# mod_syr.enableDataPandas()
mod_syr.mode_integrate_all_odes = True # As recommented by pysces
mod_syr.SetQuiet()
# C=n/V (species are µM in Prx_dim-dec)
setattr(mod_syr,
        "dimers_init", 
        inj_params["injection_dimer"]/inj_params["injection_volume"]) 
setattr(mod_syr, 
        "decamers_init", 
        inj_params["injection_decamer"]/inj_params["injection_volume"]) 

# Load first inject model
mod_first_inj = pysces.model("Prx_dim-dec.psc", dir = mod_dir)
# mod_first_inj.enableDataPandas()
mod_first_inj.mode_integrate_all_odes = True # As recommented by pysces
mod_first_inj.SetQuiet()
# C=n/V (species are µM in Prx_dim-dec)
setattr(mod_first_inj, 
        "dimers_init", 
        inj_params["injection_dimer"]/(inj_params["injection_volume"]+inj_params["cell_volume_init"]))
setattr(mod_first_inj,
        "decamers_init", 
        inj_params["injection_decamer"]/(inj_params["injection_volume"]+inj_params["cell_volume_init"])) 


In [ ]:
# Update ITC model syringe
setattr(mod_syr, "koff", getattr(mod_ITC_Prx_dim_dec,"koff"))
setattr(mod_syr, "kon",  getattr(mod_ITC_Prx_dim_dec,"kon"))
setattr(mod_syr, "exponent",  getattr(mod_ITC_Prx_dim_dec,"exponent"))

# Find equilibrium
mod_syr.doState()

# Update ITC mod syringe params (dimers and decamers in injection in AMOUNT)
# n=C*V (species are nmol in mod_ITC_Prx_dim_dec)
setattr(mod_ITC_Prx_dim_dec,  
        "injection_dimer", 
        getattr(mod_syr,"dimers_ss")*getattr(mod_ITC_Prx_dim_dec,"injection_volume"))
setattr(mod_ITC_Prx_dim_dec,
        "injection_decamer", 
        getattr(mod_syr,"decamers_ss")*getattr(mod_ITC_Prx_dim_dec,"injection_volume"))


In [ ]:
# Update ITC model with first injection
setattr(mod_first_inj, "koff", getattr(mod_ITC_Prx_dim_dec,"koff"))
setattr(mod_first_inj, "kon",  getattr(mod_ITC_Prx_dim_dec,"kon"))
setattr(mod_first_inj, "exponent",  getattr(mod_ITC_Prx_dim_dec,"exponent"))

# Find equilibrium
mod_first_inj.doState()

# Update ITC mod initial params (dimers and decamers in AMOUNT and cell volume)
# n=C*V (species are nmol in mod_ITC_Prx_dim_dec)
setattr(mod_ITC_Prx_dim_dec, 
        "dimers_init", 
        getattr(mod_first_inj,"dimers_ss")*getattr(mod_ITC_Prx_dim_dec,"cell_volume_init")) 
setattr(mod_ITC_Prx_dim_dec, 
        "decamers_init", 
        getattr(mod_first_inj,"decamers_ss")*getattr(mod_ITC_Prx_dim_dec,"cell_volume_init"))
setattr(mod_ITC_Prx_dim_dec, 
        "cell_volume_init", 
        getattr(mod_ITC_Prx_dim_dec,"injection_volume")+getattr(mod_ITC_Prx_dim_dec,"cell_volume_init"))


### Simulate ITC (figure 3a and 3b)

In [ ]:
end = 60*60 # minutes to seconds
mod_ITC_Prx_dim_dec.doSim(points = end+1,end = end)

mod_ITC_Prx_dim_dec.sim["Time (sec)"] = mod_ITC_Prx_dim_dec.sim["Time"]

df_sim = mod_ITC_Prx_dim_dec.sim
df_sim['Time (sec)'] = df_sim['Time (sec)'] - 1
df_sim = df_sim[df_sim['Time (sec)']>-0.00001]
df_sim['Time (sec)'] = df_sim['Time (sec)'].astype(int)
df_sim.set_index('Time (sec)',drop=True,inplace=True)
df_sim.head()

In [ ]:
rows = 1
cols =1

f,axarr = plt.subplots(rows,cols)
f.set_size_inches(w=mpl_width*cols, h=mpl_height*rows)

axarr.plot(df_sim.index/60, df_sim["Cdim"], label = "dimers")
axarr.plot(df_sim.index/60,df_sim["Cdec"], label = "decamers")

axarr.set_xlabel(f'Time (min)',fontsize = mpl_xlabel_fontsize)
axarr.set_ylabel(f'Species ({mu}M)',fontsize = mpl_ylabel_fontsize)
axarr.legend()

f.tight_layout()
f.savefig(os.path.join(fig_dir,"ITC_mod_species_vs_time.pdf"),dpi= mpl_dpi)

In [ ]:
rows = 1
cols = 1

f,axarr = plt.subplots(rows,cols)
f.set_size_inches(w=mpl_width*cols, h=mpl_height*rows)

axarr.plot(df_sim.index/60,
           df_sim["Cdec"]*mod_ITC_Prx_dim_dec.koff, 
           label = "Dissociation")
axarr.plot(df_sim.index/60,
           df_sim["Cdim"]**mod_ITC_Prx_dim_dec.exponent*mod_ITC_Prx_dim_dec.kon, 
           label = "Association")

axarr.set_xlabel(f'Time (min)',fontsize = mpl_xlabel_fontsize)
axarr.set_ylabel(f'Rate ({mu}M/s)',fontsize = mpl_ylabel_fontsize)
axarr.legend()

f.tight_layout()
f.savefig(os.path.join(fig_dir,"ITC_mod_rates_vs_time.pdf"),dpi= mpl_dpi)

#### Heat generation vs time (figure 3c)

In [ ]:
# Calculate enthalpy
df_sim["Rate μcal/sec"] = (-df_sim["Disassociation"])*ITC_At_inject_params["injection_enthalpy"]


In [ ]:
rows = 1
cols =1

f,axarr = plt.subplots(rows,cols)
f.set_size_inches(w=mpl_width*cols, h=mpl_height*rows)

axarr.plot(df_sim.index/60,df_sim["Rate μcal/sec"])

axarr.set_xlabel(f'Time (min)',fontsize = mpl_xlabel_fontsize)
axarr.set_ylabel(f'{mu}cal/sec',fontsize = mpl_ylabel_fontsize)

f.tight_layout()
f.savefig(os.path.join(fig_dir,"ITC_mod_heat_vs_time.pdf"),dpi= mpl_dpi)

#### Heat AUC vs protein (figure 3d)

In [ ]:
# Calculate AUC for each injection
points_per_inject = int(mod_ITC_Prx_dim_dec.injection_interval)
num_injections = math.floor(df_sim.index[-1]/points_per_inject)

inj_monomers =(mod_ITC_Prx_dim_dec.injection_decamer*10+mod_ITC_Prx_dim_dec.injection_dimer*2) # µmol monomers

heat_AUC_prot = np.zeros((num_injections,2))
start =1
for i in range(num_injections):
    x = df_sim.loc[start]["Cdim"]*2 + df_sim.loc[start]["Cdec"]*10 # µM monomers
    
    x_range = df_sim.loc[start:start+points_per_inject].index.values
    y_range = df_sim.loc[start:start+points_per_inject]["Rate μcal/sec"].values
    inj_ucal = auc(x_range,y_range) # µcal/s*s=µcal
    y = inj_ucal/inj_monomers # µcal to µcal/µmol

    heat_AUC_prot[i,0] = x
    heat_AUC_prot[i,1] = y

    start += points_per_inject


In [ ]:
rows = 1
cols = 1

f,axarr = plt.subplots(rows,cols)
f.set_size_inches(w=mpl_width*cols, h=mpl_height*rows)

axarr.plot(heat_AUC_prot[:,0],heat_AUC_prot[:,1],"o")

axarr.set_xlabel(f'Prx ({mu}M of monomer)',fontsize = mpl_xlabel_fontsize)
axarr.set_ylabel(f'kcal/mol of injectant',fontsize = mpl_ylabel_fontsize)

f.tight_layout()
f.savefig(os.path.join(fig_dir,"ITC_mod_heatAUC_vs_Prx.pdf"),dpi= mpl_dpi)

### Baseline - Interpolated 

In [ ]:
# Baseline correction function
num_disso_injections = math.floor(df_sim["Rate μcal/sec"].index[-1]/mod_ITC_Prx_dim_dec.injection_interval)

start = 1
end = mod_ITC_Prx_dim_dec.injection_interval

df_temp = pd.DataFrame(df_sim["Rate μcal/sec"].loc[start: end])

next_baseline = df_temp.iloc[-10:-5]
print(next_baseline.index.values.mean())

In [ ]:
# Baseline correction function
num_disso_injections = math.floor(df_sim["Rate μcal/sec"].index[-1]/mod_ITC_Prx_dim_dec.injection_interval)

start = 1
end = mod_ITC_Prx_dim_dec.injection_interval

df_temp = pd.DataFrame(df_sim["Rate μcal/sec"].loc[start: end])

next_baseline = df_temp.iloc[-10:-5]

x2 = next_baseline.index.values.mean()
y2 = next_baseline["Rate μcal/sec"].mean()

x1 = 0.0
y1 = next_baseline["Rate μcal/sec"].mean()

baseline_interp = interp1d([x1,x2], [y1,y2],fill_value="extrapolate")(df_temp.index)
plt.plot(df_temp.index,baseline_interp)

current_baseline = next_baseline

df_temp["Rate μcal/sec"] = df_temp["Rate μcal/sec"] - baseline_interp

df_sim_baseline = pd.DataFrame(df_temp)

count = 1
while count < num_disso_injections:
    start = count*mod_ITC_Prx_dim_dec.injection_interval+1
    end = count*mod_ITC_Prx_dim_dec.injection_interval + mod_ITC_Prx_dim_dec.injection_interval

    df_temp = pd.DataFrame(df_sim["Rate μcal/sec"].loc[start: end])

    next_baseline = df_temp.iloc[-10:-5]

    x2 = next_baseline.index.values.mean()
    y2 = next_baseline["Rate μcal/sec"].mean()

    x1 = current_baseline.index.values.mean()
    y1 = current_baseline["Rate μcal/sec"].mean()

    baseline_interp = interp1d([x1,x2], [y1,y2],fill_value="extrapolate")(df_temp.index)

    df_temp["Rate μcal/sec"] = df_temp["Rate μcal/sec"] - baseline_interp
    plt.plot(df_temp.index,baseline_interp)

    current_baseline = next_baseline

    df_sim_baseline = pd.concat([df_sim_baseline,df_temp])
    
    count += 1

df_sim["Rate μcal/sec baseline"] = df_sim_baseline["Rate μcal/sec"]


#### Heat generation vs time (figure 3e)

In [ ]:
rows = 1
cols =1

f,axarr = plt.subplots(rows,cols)
f.set_size_inches(w=mpl_width*cols, h=mpl_height*rows)

axarr.plot(df_sim.index/60,df_sim["Rate μcal/sec baseline"],label = "Baseline corrected data")
axarr.plot(df_sim.index/60,df_sim["Rate μcal/sec"],'k--',label = "Non-corrected data")

axarr.set_xlabel(f'Time (min)',fontsize = mpl_xlabel_fontsize)
axarr.set_ylabel(f'{mu}cal/sec',fontsize = mpl_ylabel_fontsize)
axarr.set_title("Barranco-Medina Fig 1a top")

f.tight_layout()

In [ ]:
rows = 1
cols =1

f,axarr = plt.subplots(rows,cols)
f.set_size_inches(w=mpl_width*cols, h=mpl_height*rows)

axarr.plot(df_sim.index/60,df_sim["Rate μcal/sec baseline"],label = "Baseline corrected data")

axarr.set_xlabel(f'Time (min)',fontsize = mpl_xlabel_fontsize)
axarr.set_ylabel(f'{mu}cal/sec',fontsize = mpl_ylabel_fontsize)

f.tight_layout()
f.savefig(os.path.join(fig_dir,"ITC_mod_baseline_corr_heat_vs_time.pdf"),dpi= mpl_dpi)

#### Heat AUC vs protein (figure 3f)

In [ ]:
# Calculate AUC for each injection
points_per_inject = int(mod_ITC_Prx_dim_dec.injection_interval)
num_injections = math.floor(df_sim.index[-1]/points_per_inject)

inj_monomers =(mod_ITC_Prx_dim_dec.injection_decamer*10+mod_ITC_Prx_dim_dec.injection_dimer*2)

heat_AUC_prot = np.zeros((num_injections,2))
start =0
for i in range(num_injections):
    x = df_sim.loc[start]["Cdim"]*2 + df_sim.loc[start]["Cdec"]*10
    
    x_range = df_sim.loc[start:start+points_per_inject].index.values
    y_range = df_sim.loc[start:start+points_per_inject]["Rate μcal/sec baseline"].values
    inj_ucal = auc(x_range,y_range) # µcal/s*s=µcal
    y = inj_ucal/inj_monomers # µcal to µcal/µmol
    
    heat_AUC_prot[i,0] = x
    heat_AUC_prot[i,1] = y

    start += points_per_inject

In [ ]:
rows = 1
cols = 1

f,axarr = plt.subplots(rows,cols)
f.set_size_inches(w=mpl_width*cols, h=mpl_height*rows)

axarr.plot(heat_AUC_prot[:,0],heat_AUC_prot[:,1],"o")

axarr.set_xlabel(f'Prx ({mu}M of monomer)',fontsize = mpl_xlabel_fontsize)
axarr.set_ylabel(f'kcal/mol of injectant',fontsize = mpl_ylabel_fontsize)

f.tight_layout()
f.savefig(os.path.join(fig_dir,"ITC_mod_baseline_corr_heatAUC_vs_Prx.pdf"),dpi= mpl_dpi)